## Micrograd / Backpropagation

What are neural nets
- mathematical expressions
- fairly simple in the case of MLP
- take input:
  - data
  - weights&params
- forward pass => loss function, which tries to measure the accuracy of the predictions, loss function is low when the NN is doing what you want it to do
- use backpropagation to get the gradient, then we know how to tune all the parameters to decrease the loss locally
- iterate that process many times: "gradient descent"
- we have a blob of neural stuff ("simulated neural tissue") and we can make it do arbitrary things
- these neural nets have fascinating emergent properties
- this NN is 41 parameters. GPT is hundreds of billions, but works fundamentally in the same way.
- real one:
  - not mean squared error, but cross entropy loss (or max margin, binary cross entropy)
  - use a different method of updating
  - micrograd uses relu instead of tanh
  - loss function usually only run on a subset of the data per pass
  - "L2 Regularization" => prevents overfitting
  - "learning rate decay" => start high, become lower over time

In [1]:
import sys

!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install torch torchvision

import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


## Value Class

The point of this is that each value object tracks its **dependencies** ("i was created by adding 2 + 5"), and the **gradient** ("the current slope of this dependency indicates that increasing it would have xyz change on me", basically a derivative). We need this so that we can backtrack.

In [11]:
# wraps a value
class Value:
    # `()` is an empty tuple
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        # derivative of L with respect to this value
        # 0 means "no effect" (gradient 0)
        self.grad = 0.0
        # default to nothing, on a leaf node there is nothing to do
        self._backward = lambda: None
        self._prev = set(_children) # set is cheaper than tuple
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # every grad needs to be multiplied by out.grad to respect the chain rule
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
 
        return out
        
    # 2.__dd__(Value(3.0)) doesn't work
    # so python checks if Value has an `radd` that knows how to do it
    # "reverse add"
    def __radd__(self, other):
        return self + other

    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out

    def __rmul__(self, other): 
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad
        out._backward = _backward

        return out
        
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            # derivative of e^x is e^x
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    # we could implement `__exp__` instead... but want to show here
    # that we can build functiond at arbitrary places in the complexity
    # stack
    # !! AS LONG AS WE KNOW THE LOCAL DERIVATIVE !!
    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1)/(math.exp(2*n) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                # each node only adds itself to the list
                # after all of its children have been processed
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0

        for node in reversed(topo):
            node._backward()

![Neuron](./_resources/neuron.png)

A Neuron
- Also called Perceptron, which is an artificial neuron / mathematical equation: `y = f(w1x1 + w2x2 + ... + wnxn + b)`
- takes one or more numerical inputs (x1, x2, ..., xn)
- multiplies each input by a weight (w1, w2, ..., wn), which are parameters
- takes the sum of these x1w1 + x2w2 + ... + xnwn
- adds the bias, which is the parameter
- runs that through an activation function like tanh or relu
- passes the outcome to one or more neurons in the next layer (this is called Multi Layer Perceptron)

In [3]:
# a single neuron that subscribes to the pytorch api

class Neuron:
    # nin: number of inputs - w0x0, w1x1, etc
    def __init__(self, nin):
        # weight
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        # bias
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        # w * x + b
        # multiply all the elements of w with all the elements of x, pair wise
        # raw activation
        act = sum(wi * xi for wi, xi in zip(self.w, x)) + self.b
        # pass that through a nonlinearity
        out = act.tanh()
        return out

    # same name as in pytorch
    def parameters(self):
        return self.w + [self.b]

![Neural Net](./_resources/neural_net2.jpg)

Real LLMs can have much larger output layers. Sometimes (eg binary classification), one neuron is enough.

In the case of something like GPT-4o, there are thousands of neurons in the output layer, each one contains the likelihood of that token. ie for input "the cat sat on the":

"mat" → 0.75, "floor" → 0.10, "table" → 0.05, "moon" → 0.01, ...

Then it either picks the highest-probability token, or adds some randomness through techniques like temperature sampling

If generating 100 tokens and it has a vocabulary of 100k, the output would be a 100 x 100k matrix

In [9]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        # long form version:
        # params = []
        # for neuron in self.neurons
        #     ps = neuron.parameters()
        #     params.extend(ps)
        # return params

In [5]:
# multi layer perceptron
class MLP:
    # nouts is a list, defines the sizes of all the layers
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [10]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

n = MLP(3, [4,4,1])

step = 0

# gradient descent

learning_rate = 0.05 # if we make too large of a step, we may overstep...

print("targets", ys)

for k in range(20):
    step += 1

    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout-ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0  # Reset gradients to avoid accumulation (all the operations on p.grad are +=)
    
    loss.backward()

    # update
    for p in n.parameters():
        # -1 because we want to minimize the loss, not maximize it
        # The gradient of a loss function L with respect to a parameter w (i.e., dL/dw)
        # is a vector pointing in the direction that increases the loss the most.
        # "the gradient vectors are pointing in the direction of increasing the loss"
        p.data += learning_rate * p.grad * -1

    print("step", step)
    print("    curr:", [value.data for value in ypred])
    print("    loss:", loss.data)

targets [1.0, -1.0, -1.0, 1.0]
step 1
    curr: [-0.9270016790755088, -0.8928533274122414, -0.848556003804691, -0.9116328426552154]
    loss: 7.402091289708123
step 2
    curr: [-0.8818482070283263, -0.8382348497473819, -0.7887388201806861, -0.8581389571556461]
    loss: 7.064832308330099
step 3
    curr: [-0.7657897598497311, -0.7131426296153756, -0.6649362844291296, -0.7237808372088818]
    loss: 6.283988695154863
step 4
    curr: [-0.39319773622744664, -0.3770748814142159, -0.3720276793003758, -0.3164821078196889]
    loss: 4.456510011368637
step 5
    curr: [0.264256911208705, 0.11794358616453098, 0.023020382085314275, 0.3419865949516506]
    loss: 3.2706680979358733
step 6
    curr: [0.08886614994099246, -0.19527525648188238, -0.2945303953362878, 0.22420666928171679]
    loss: 2.577289460644992
step 7
    curr: [0.41547017612471754, -0.12164858290395585, -0.2894446358608733, 0.5477658610611773]
    loss: 1.822580968842923
step 8
    curr: [0.39882822518762157, -0.4172037509907331,